## Punto dos Laboratorio 5 MOS
- Samuel Ramirez Gómez
- Paulina Arrazzola

# Conjuntos 
- Recursos
- Zona Destino
- Viajes
- Avion


# Indices

In [20]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import os
numav = 4
numrec = 5
numzonas = 4
numviajes = 2
model = ConcreteModel()
model.R = RangeSet(numrec)
model.V = RangeSet(numviajes)
model.Z = RangeSet(numzonas)
model.A = RangeSet(numav)

# Parametros

In [21]:
# Recursos
Impacto = {1:50,2:100,3:120,4:60,5:40}
Peso = {1:5,2:2,3:0.3,4:6,5:3}
Volumen = {1:3,2:1,3:0.5,4:4,5:2}
Disponibilidad = {1:12,2:15,3:40,4:15,5:20}
model.I = Param(model.R,initialize=Impacto)
model.P = Param(model.R,initialize=Peso)
model.Vo = Param(model.R,initialize=Volumen)
model.Di = Param(model.R,initialize=Disponibilidad)

#Aviones 
capacidadp = {1:40,2:50,3:60,4:45}
capacidadv = {1:35,2:40,3:45,4:38}
costofijo = {1:15,2:20,3:25,4:18}
costovar = {1:0.020,2:0.025,3:0.03,4:0.022}
model.C = Param(model.A,initialize=capacidadp)
model.Cv = Param(model.A,initialize=capacidadv)
model.F = Param(model.A,initialize=costofijo)
model.K = Param(model.A,initialize=costovar)

#Zonas
distancia = {1:800,2:1200,3:1500,4:900}
multiplicador = {1:1.2,2:1.5,3:1.8,4:1.4}
model.D = Param(model.Z,initialize=distancia)
model.Mi = Param(model.Z,initialize=multiplicador)

#Matriz necesidades
necesidades = {(1,1):8,(1,2):6,(1,3):2,(1,4):0.6,(1,5):3,
               (2,1):12,(2,2):9,(2,3):3,(2,4):0.9,(2,5):5,
                (3,1):16,(3,2):12,(3,3):4,(3,4):1.2,(3,5):7,
                 (4,1):10,(4,2):8,(4,3):2,(4,4):0.6,(4,5):4 }
model.N = Param(model.Z,model.R,initialize=necesidades)



# Variables de decisión

In [22]:
model.x = Var(model.R,model.A,model.Z,model.V, within=PositiveReals)
model.y= Var(model.A,model.V,model.Z, within= Binary)
model.u = Var(model.A,within=Binary)
model.me = Var(model.A,model.Z,model.V,within=PositiveIntegers)


# Funciones objetivo

In [23]:
model.obj = Objective(expr= sum(model.x[r,a,d,v]*model.I[r]*model.Mi[d] for r in model.R for a in model.A for d in model.Z for v in model.V),sense=maximize)
#model.obj = Objective(expr = sum(model.u[a]*model.F[a] for a in model.A) + sum(model.y[a,v,d]*model.D[a]*model.K[a] for a in model.A for v in model.V for d in model.Z),sense=minimize)

# Restricciones

In [24]:
model.restricciones = ConstraintList()
for a in model.A:
    for d in model.Z:
        for v in model.V:
           model.restricciones.add(sum(model.x[r,a,d,v]*model.P[r] for r in model.R) <= model.C[a])
           model.restricciones.add(sum(model.x[r,a,d,v]*model.Vo[r] for r in model.R) <= model.Cv[a])


for d in model.Z:
    for r in model.R:
        model.restricciones.add(sum(model.x[r,a,d,v]*model.P[r] for v in model.V for a in model.A) >= model.N[d,r])

for r in model.R:
    model.restricciones.add(sum(model.x[r,a,d,v]for a in model.A for d in model.D for v in model.V) <= model.Di[r])

model.restricciones.add(sum(model.x[3,1,d,v] for d in model.Z for v in model.V) == 0)

for a in model.A:
    for v in model.V:
        model.restricciones.add(model.x[3,a,d,v] + model.x[3,a,d,v] <= 1)
        model.restricciones.add(sum(model.y[a,v,d] for d in model.Z) == 1)

for a in model.A:
    for v in model.V:
        for d in model.Z:
           model.restricciones.add(model.x[3,a,d,v,] == model.me[a,d,v])
           for r in model.R:
             model.restricciones.add(model.x[r,a,d,v] == model.y[a,v,d])






